In [1]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

In [ ]:
model = load_model('gender_detection.model')


webcam = cv2.VideoCapture(0)
    
classes = ['man','woman']

while webcam.isOpened():
    
    # HEATMAP

#     # read frame from webcam 
#     status, frame = webcam.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # apply face detection
#     face, confidence = cv.detect_face(frame)
    
#     # Generate the heatmap
#     heatmap = np.zeros_like(gray, dtype=np.float32)
#     for idx, f in enumerate(face):
#         (startX, startY) = f[0], f[1]
#         (endX, endY) = f[2], f[3]
#         heatmap[startY:endY, startX:endX] += 1.0
        
#     #normalise the heatmap
#     heatmap = cv2.normalize(heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
#     # Apply the colormap
#     heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    
#     # Overlay the heatmap on top of the original frame
#     result = cv2.addWeighted(frame, 0.7, heatmap, 0.3, 0)
    
#     # Display the resulting frame
#     cv2.imshow('Face Detection Heatmap', result)
    
#     # Break the loop if 'q' is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
    
    # GENDER DETECTOR
    
     #read frame from webcam 
    status, frame = webcam.read()
    
    face, confidence = cv.detect_face(frame)
    for idx, f in enumerate(face):

           
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)


        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0]

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = startY - 10 if startY - 10 > 10 else startY + 10


        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)


    cv2.imshow("gender detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


webcam.release()
cv2.destroyAllWindows()

1/1 [==============================] - ETA: 0s